In [ ]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import datetime
import cvxpy as cp
from pypfopt import EfficientFrontier
from pypfopt import EfficientCVaR
from pypfopt.plotting import plot_efficient_frontier


In [ ]:
def variance(w, cov):
    return np.dot(w.T, np.dot(cov,w)) / (w.shape[0]-1)

def returns(w, exp):
    return np.dot(w.T, exp) / (w.shape[0]-1)

# Process Data

In [ ]:
DATA_PATH = '../data'
PRICES_PATH = os.path.join(DATA_PATH, 'prices')
SIMULATION_DATA_PATH = os.path.join(DATA_PATH, 'simulation data')
prices = pd.read_csv(os.path.join(PRICES_PATH, 'Spain_prices.csv'))

In [ ]:
prices['price'] = prices['Price (EUR/MWhe)']
prices['time'] = pd.to_datetime(prices['Datetime (UTC)'])
prices.head()

In [ ]:
df_list = []
for file_name in os.listdir(SIMULATION_DATA_PATH):
    df = pd.read_csv(os.path.join(SIMULATION_DATA_PATH, file_name),sep = ',', comment = '#')
    file_name_split = file_name.split('_')
    df['lat_lon'] = file_name_split[2]+ '_' + file_name_split[3]
    df_list.append(df)
production = pd.concat(df_list)

In [ ]:
production['time'] = pd.to_datetime(production['time'])
production.head()

In [ ]:
production['hour'] = production['time'].dt.hour
production['day'] = production['time'].dt.day
production['month'] = production['time'].dt.month
production['year'] = production['time'].dt.year

prices['hour'] = prices['time'].dt.hour
prices['day'] = prices['time'].dt.day
prices['month'] = prices['time'].dt.month
prices['year'] = prices['time'].dt.year

In [ ]:
# Filter desired dates
final_time_price = prices['time'].max()
start_time_price = final_time_price - pd.Timedelta(days=365)

mask_dates = (prices['time'] <= final_time_price) & (prices['time'] > start_time_price)
last_y_prices = prices[mask_dates]
last_y_prices.head()

In [ ]:
production.head()

In [ ]:
# Merge in same day but different year and compute revenue
df_revenue = pd.merge(production, last_y_prices, on=['hour','day','month'], how='inner')
df_revenue = df_revenue[['lat_lon','price','electricity','hour','day','month']]
df_revenue['revenue'] = df_revenue['price'] * df_revenue['electricity'] / 1000 # (price is in EUR/MWh and electricity is in kWh)

In [ ]:
# Add fictitious year and time stamp so we have a time series
df_revenue['year'] = 2019
df_revenue['time_stamp'] = df_revenue[['year', 'month', 'day', 'hour']].apply(lambda s : datetime.datetime(*s),axis = 1)

## Start model

In [ ]:
monthly_rev = pd.pivot_table(df_revenue, values='revenue', index='time_stamp', columns='lat_lon', aggfunc='sum')
monthly_rev

In [ ]:
cov = monthly_rev.cov()
exp = monthly_rev.mean()   # mean() ¿
#monthly_rev.sum()
# Hourly correlation is still very big although some points close to 0.5
monthly_rev.corr()

In [ ]:
# Generate uniform random weights and plot
n_samples = 1000
n_locations = monthly_rev.shape[1]

w_np = np.random.uniform(size=(n_samples, n_locations))
w_np /= w_np.sum(axis=1, keepdims=True)
df_w = pd.DataFrame(w_np, columns=monthly_rev.columns)

df_plot = pd.DataFrame(data = [], index=df_w.index)
df_plot['cov'] =  df_w.apply(variance, axis=1, cov=cov)
df_plot['exp'] = df_w.apply(returns, axis=1, exp=exp)

plt.scatter(x=df_plot['cov'], y=df_plot['exp'])
plt.show()

In [ ]:
# Compute 3 special points (min var, max sharpe, max returns)

ef = EfficientFrontier(exp, cov, weight_bounds=(0,1))
min_volatility_w = pd.Series((ef.min_volatility())) # ef.clean_weights()

ef = EfficientFrontier(exp, cov, weight_bounds=(0,1))
max_sharpe_w = pd.Series((ef.max_sharpe()))

max_returns_location = monthly_rev.sum().idxmax()
max_returns_w = {location: float(0) for location in max_sharpe_w.index}
max_returns_w[max_returns_location] = 1.0
max_returns_w = pd.Series(max_returns_w)

extremes_w = [min_volatility_w, max_sharpe_w, max_returns_w]
extremes_var = [variance(w, cov) for w in extremes_w]
extremes_exp = [returns(w, exp) for w in extremes_w]

In [ ]:
# Compute points in boundary

boundary_range_exp = np.linspace(min(extremes_exp)+0.001, max(extremes_exp)-0.001, 100)
boundary_w = []

for boundary_exp in boundary_range_exp:
    ef = EfficientFrontier(exp, cov, weight_bounds=(0,1))
    w = pd.Series(ef.efficient_return(boundary_exp))
    boundary_w.append(w)

boundary_var = [variance(w, cov) for w in boundary_w]
boundary_exp = [returns(w, exp) for w in boundary_w]

In [ ]:
# Now we get more resonable picture TODO: see which linear convinations we get the blue curve.
ef = EfficientFrontier(exp, cov,  weight_bounds=(0,1))
fig, ax = plt.subplots()
plot_efficient_frontier(ef, ax=ax, show_assets=True)
plt.scatter(x=np.sqrt(df_plot['cov']), y=df_plot['exp'])
plt.scatter(np.sqrt(extremes_var), extremes_exp)
plt.show()

In [ ]:
# TODO:
# - Get 3 special points
# - Get curve
# - Plot solutions on map nicely.
# -
# - compare minimum vairiance/sharpe with what one would get in specific location. How much improvement
# - Look correlations and see if we can get good result with fewer assets than boundary?
#
#  utilitzar els 10 anys de preus? Fer model treient dades de vent reals i produccio?

In [ ]:
#TODO: mirar si algunes estacions mai utilitzades en corva.